# Importing Necessary Packages

In [5]:
import csv
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import seaborn as sns

from keras.models import Sequential,Model
from keras.models import Model
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Embedding
from keras.layers import Lambda
from keras.layers import Dense, Activation
from keras.layers import Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from matplotlib import pyplot
from keras.layers.merge import concatenate

from keras import backend as K

K.set_image_dim_ordering('th')

## Neutral Network Building

In [2]:
from keras.utils import plot_model
import pydot, graphviz

# Variational Autoencoder

In [8]:
#-----------------------------------------------------------------------------
# Define Sampling Layer
#-----------------------------------------------------------------------------

def sampling(args):
    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch,dim))#, mean=0., std=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

batch_size = 128
latent_dim = 3

input_img = Input(shape = (1,20,20))

layer_1 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(input_img)
layer_2 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(layer_1)
pooling = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(layer_2)
layer_3 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(pooling)
layer_4 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(layer_3)
flatten = Flatten()(layer_4)

dense_1 = Dense(units=100, activation='relu')(flatten)
dense_2 = Dense(units=50, activation='relu')(dense_1)

# Latent space
z_mean = Dense(units=latent_dim)(dense_2)
z_log_sigma = Dense(units=latent_dim)(dense_2)

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean,z_log_sigma])

# Decoder
dense_3 = Dense(units=50, activation='relu')(z)
dense_4 = Dense(units=100, activation='relu')(dense_3)
reshape = Reshape((1,10,10))(dense_4)
layer_5 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(reshape)
layer_6 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(layer_5)
upsample = UpSampling2D((2,2))(layer_6)
layer_7 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(upsample)

decoded = Conv2D(1, kernel_size=4,padding="same", activation = 'softmax')(layer_7)


vae = Model(input_img, decoded)
#vae.summary() # show summary of the network

plot_model(vae, show_shapes=True, show_layer_names=True,to_file='VAE.pdf')

# Autoencoder

In [5]:
input_img = Input(shape = (1,20,20))

layer_1 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(input_img)
layer_2 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(layer_1)
pooling = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(layer_2)
layer_3 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(pooling)
layer_4 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(layer_3)
flatten = Flatten()(layer_4)

dense_1 = Dense(units=100, activation='relu')(flatten)
dense_2 = Dense(units=50, activation='relu')(dense_1)
encoded = Dense(units=20)(dense_2)

# Decoder
dense_3 = Dense(units=50, activation='relu')(encoded)
dense_4 = Dense(units=100, activation='relu')(dense_3)
reshape = Reshape((1,10,10))(dense_4)
layer_5 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(reshape)
layer_6 = Conv2D(5, kernel_size=2,padding="same", activation = 'relu')(layer_5)
upsample = UpSampling2D((2,2))(layer_6)
layer_7 = Conv2D(10, kernel_size=4,padding="same", activation = 'relu')(upsample)

decoded = Conv2D(1, kernel_size=4,padding="same", activation = 'softmax')(layer_7)

autoencoder = Model(input_img, decoded)
#autoencoder.summary()

plot_model(autoencoder, show_shapes=True, show_layer_names=True,to_file='autoencoder.png')

# CNN 1

In [6]:
# Alternative way to make sequential (not ideal)
model = Sequential()
model.add(Conv2D(20, kernel_size=4,input_shape=(1, 20, 20), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))
#model.add(Conv2D(40, kernel_size=4, padding="same", activation = 'relu'))
model.add(Conv2D(40, kernel_size=4, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=3, activation = 'softmax'))

#model.summary()
plot_model(model, show_shapes=True, show_layer_names=True,to_file='CNN_1.png')

# ResNet

In [7]:

# Channel 1
input_img = Input(shape = (1,20,20))
input_cnn = Conv2D(5, kernel_size=4,padding="same", activation = 'relu')(input_img)

# ResNet Unit 1 - 1
d1p1_1 = Conv2D(10, kernel_size=4,padding="same")(input_cnn)
d1p1_1_norm = BatchNormalization()(d1p1_1)
d1p1_activ = Activation('relu')(d1p1_1_norm)
d1p1_2 = Conv2D(5, kernel_size=4,padding="same")(d1p1_activ)
d1p1_2_norm = BatchNormalization()(d1p1_2)

# ResNet Unit 1 - 2
d1p2_1 = Conv2D(5, kernel_size=4,padding="same")(input_cnn)
d1p2_1_norm = BatchNormalization()(d1p2_1)
d1p2_activ = Activation('relu')(d1p2_1_norm)
d1p2_2 = Conv2D(5, kernel_size=4,padding="same")(d1p2_activ)
d1p2_2_norm = BatchNormalization()(d1p2_2)

# Add layers
d1_sum = keras.layers.add([input_cnn, d1p1_2_norm, d1p2_2_norm])
d1_sum_activ = Activation('relu')(d1_sum)


# ResNet Unit 2 - 1
d2p1_1 = Conv2D(10, kernel_size=4,padding="same")(d1_sum_activ)
d2p1_1_norm = BatchNormalization()(d2p1_1)
d2p1_activ = Activation('relu')(d2p1_1_norm)
d2p1_2 = Conv2D(5, kernel_size=4,padding="same")(d2p1_activ)
d2p1_2_norm = BatchNormalization()(d2p1_2)

# ResNet Unit 2 - 2
d2p2_1 = Conv2D(5, kernel_size=4,padding="same")(d1_sum_activ)
d2p2_1_norm = BatchNormalization()(d2p2_1)
d2p2_activ = Activation('relu')(d2p2_1_norm)
d2p2_2 = Conv2D(5, kernel_size=4,padding="same")(d2p2_activ)
d2p2_2_norm = BatchNormalization()(d2p2_2)

# Add layers
d2_sum = keras.layers.add([d1_sum_activ, d2p1_2_norm, d2p2_2_norm])
d2_sum_activ = Activation('relu')(d2_sum)


# ResNet Unit 3 - 1
d3p1_1 = Conv2D(10, kernel_size=4,padding="same")(d2_sum_activ)
d3p1_1_norm = BatchNormalization()(d3p1_1)
d3p1_activ = Activation('relu')(d3p1_1_norm)
d3p1_2 = Conv2D(5, kernel_size=4,padding="same")(d3p1_activ)
d3p1_2_norm = BatchNormalization()(d3p1_2)

# ResNet Unit 3 - 2
d3p2_1 = Conv2D(5, kernel_size=4,padding="same")(d2_sum_activ)
d3p2_1_norm = BatchNormalization()(d3p2_1)
d3p2_activ = Activation('relu')(d3p2_1_norm)
d3p2_2 = Conv2D(5, kernel_size=4,padding="same")(d3p2_activ)
d3p2_2_norm = BatchNormalization()(d3p2_2)

# Add layers
d3_sum = keras.layers.add([d2_sum_activ, d3p1_2_norm, d3p2_2_norm])
d3_sum_activ = Activation('relu')(d3_sum)


# Down sampling 
d4down = Conv2D(10, kernel_size=4,strides=2,padding="same")(d3_sum_activ)
d4down_norm = BatchNormalization()(d4down)

# ResNet Unit 4 - 1
d4p1_1 = Conv2D(20, kernel_size=4,strides=2,padding="same")(d3_sum_activ)
d4p1_1_norm = BatchNormalization()(d4p1_1)
d4p1_activ = Activation('relu')(d4p1_1_norm)
d4p1_2 = Conv2D(10, kernel_size=4,padding="same")(d4p1_activ)
d4p1_2_norm = BatchNormalization()(d4p1_2)

# ResNet Unit 4 - 2
d4p2_1 = Conv2D(10, kernel_size=4,strides=2,padding="same")(d3_sum_activ)
d4p2_1_norm = BatchNormalization()(d4p2_1)
d4p2_activ = Activation('relu')(d4p2_1_norm)
d4p2_2 = Conv2D(10, kernel_size=4,padding="same")(d4p2_activ)
d4p2_2_norm = BatchNormalization()(d4p2_2)

# Add layers
d4_sum = keras.layers.add([d4down_norm, d4p1_2_norm, d4p2_2_norm])
d4_sum_activ = Activation('relu')(d4_sum)

flatten = Flatten()(d4_sum_activ)

dense_1 = Dense(units=200, activation='relu')(flatten)
dense_2 = Dense(units=100, activation='relu')(dense_1)
classify = Dense(units=3, activation='softmax')(dense_2)


ResNet = Model(input_img, classify)
plot_model(ResNet, show_shapes=True, show_layer_names=True,to_file='resnet_model.png')
#print ResNet.summary()

# CNN 2

In [4]:
# Better way to CNN
input_img = Input(shape = (1,20,20))
input_cnn = Conv2D(20, kernel_size=4,padding="same", activation = 'relu')(input_img)
maxpool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(input_cnn)
dropout1 = Dropout(0.3)(maxpool1)
conv1 = Conv2D(40, kernel_size=4, padding="same", activation = 'relu')(dropout1)
maxpool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)
dropout2 = Dropout(0.3)(maxpool2)

flat = Flatten()(dropout2)

dense1 = Dense(units=300, activation='relu')(flat)
dropout3 = Dropout(0.3)(dense1)
dense2 = Dense(units=100, activation='relu')(dropout3)
dense3 = Dense(units=100, activation='relu')(dense2)
final = Dense(units=1, activation = 'sigmoid')(dense3)


CNNmodel = Model(input_img, final)

# plot_model not working due to some $PATH problem
plot_model(CNNmodel, show_shapes=True, show_layer_names=True,to_file='CNN_model.pdf')

# How to Optimize

In [ ]:
CNNmodel.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy','binary_crossentropy'])

# enable early stopping based on mean_squared_error
earlystopping = EarlyStopping(monitor="binary_crossentropy", patience=20, verbose=1, mode='auto')


# Reduce learning rate(lr) when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,
                             patience=10, min_lr = 0.0001) 

history = model.fit(X_train, Y_train, epochs=400, verbose=2, 
                      batch_size=32, callbacks=[reduce_lr,earlystopping], validation_data=(X_test, Y_test))